In [128]:
import pandas as pd 

data = pd.read_csv("data.csv", header = None)
data.head()

X = data.loc[:, 0:4]
y = data.loc[:, 5]

X = (X - X.mean())/X.std()

X = np.array(X)
y = np.array(y)

In [129]:
def poly_comb(features, deg):
    all_combs = []
    all_combs.append(list(features))
    chunks = np.arange(len(features))
    
    for i in range(0, deg-1):
        deg_combs = []
        for j in range(len(features)):
            next_chunk = len(deg_combs)
            for l in range(chunks[j], len(all_combs[i])):
                deg_combs.append(features[j]*all_combs[i][l])
            chunks[j] = next_chunk
        all_combs.append(deg_combs)
    
    all_combs = [j for i in all_combs for j in i]
    return all_combs

In [130]:
X_poly = []
for i in range(X.shape[0]):
    X_poly.append(poly_comb(X[i], 5))

#print(X_poly)
#X_poly = np.where(np.array(X_poly) < 1e-5, 0, np.array(X_poly))

In [117]:
len(X_poly[0])

251

In [131]:
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(np.array(X_poly), y, test_size=0.33, random_state=42)
X_train = np.c_[np.ones(len(X_train),dtype='int64'),X_train]
X_test = np.c_[np.ones(len(X_test),dtype='int64'),X_test]

In [197]:
class My_LinearRegression():
    def my_fit(self, X, y, iterations = 10000, alpha = 0.03, t0 = 5, t1 = 50, method = 'batch'):
        self.t0 = t0
        self.t1 = t1
        if method == 'batch':
            self.batch_gradient_descent(X, y, alpha, iterations)
        elif method == 'stohastic':
            self.stochastic_gradient_descent(X, y, iterations)
            
    def pred(self, X):
        return X.dot(self.theta)
    
    def batch_gradient_descent(self, X, Y, alpha, iterations):
        #theta = np.zeros(X_train.shape[1])
        theta = np.random.rand(X_train.shape[1])
        m = len(Y)
        cost_history = [0] * iterations
        alpha2 = alpha/100
        al = np.linspace(alpha, alpha2, iterations)
        for iteration in range(iterations):
            change = al[iteration] * X.T.dot(X.dot(theta) - Y) / m
            change = np.where(change > 100, 100, change)
            change = np.where(change < -100, -100, change)
            theta = theta - change
            #print(theta)
            #cost = cost_function(X, Y, theta)
            cost_history[iteration] = np.sum((X.dot(theta) - Y) ** 2)/(2 * m)
        self.theta = theta 
        self.cost_history = cost_history
    
    def learning_schedule(self, t):
        return self.t0/(t + self.t1)

    def stochastic_gradient_descent(self, X, y, iterations):
        theta = np.random.randn(X_train.shape[1])
        m = X.shape[0]
        for iter_ in range(iterations):
            for i in range(X.shape[0]):
                random_index = np.random.randint(m)
                xi = X[random_index:random_index+1]
                yi = y[random_index:random_index+1]
                gradients = 2*xi.T.dot(xi.dot(theta) - yi)
                eta = self.learning_schedule(iter_*m + i)
                theta = theta - eta*gradients
                
        self.theta = theta
        
def my_mse(y_pred, y_test):
    return np.sum((y_pred - y_test)**2)/len(y_pred)

In [198]:
my_lr = My_LinearRegression()

In [199]:
my_lr.my_fit(X_train, y_train)
y_pred = my_lr.pred(X_test)
print(my_mse(y_pred, y_test))

40962846.14146806


In [200]:
my_lr.cost_history

[368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,
 368453419.1610251,
 931241512.5456415,


In [138]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(np.array(X_poly), y, test_size=0.33, random_state=42)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

mean_squared_error(y_test, y_pred)

29855819.876150638